# Scratch work for EDA of axon fragment imaging

As part of Mayoral lab investigation of oligodendrocyte precursor cell influence on neurodegenertation, have fluroescent images with channels:
- GFP (sparse axonal labeling)
- DegenoTag
- Myelin staining
- DAPI

Goal is to development automated pipelines for assessing degree of degeration across different samples. Initial goal is to develop algorithms for detecting and statistically characterizing fragments of axons.

In [19]:
#%% Import block

# System
import os
import os.path


# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt 

# CSV Handling
import csv

# Image handling
import tifffile as tf 
import imageio as img # tiff writing
import czifile
from tifffile import imsave, imread, imwrite

# Numerical and statistics
import seaborn as sb
import pandas as pd 
import numpy as np
from PIL import Image
import math


from scipy.signal import find_peaks, peak_prominences
from scipy.stats import mode
from scipy.ndimage import gaussian_filter

# Image analysis
import cv2 as cv
import skimage as sk
from skimage import io, morphology
from skimage.morphology import remove_small_objects
from skimage.color import gray2rgb
from skimage.measure import label, regionprops
from skimage import data
from skimage import img_as_float
from skimage.morphology import reconstruction

# CSV Parsing Cell
## The purpose of the cell below is to
- Read the CSV Files
- Parse the CSV Files
- Extract statistical information from the fragment lengths.


In [20]:


roi_directory = 'data/ROI'
# List all CSV files in the directory
csv_files = [f for f in os.listdir(roi_directory) if f.endswith('.csv')]
# Loop through each CSV file and read its contents
total_lengths_array = []
for csv_file in csv_files:
  csv_lengths_array = []
  file_path = os.path.join(roi_directory, csv_file)
  with open(file_path, 'r') as file:
      csvreader = csv.reader(file)
      print(f"Contents of {csv_file}:")
      header = next(csvreader)
      for row in csvreader:
        csv_lengths_array.append(row[5])
      # print(row[5])
      # print(csv_lengths_array)
      numeric_values = [float(num) for num in csv_lengths_array]
      print(numeric_values)
      total_lengths_array.append(numeric_values)
  print("\n")  # Add a newline for better readability between files

Contents of LH_1.csv:
[84.624, 79.256, 68.763, 38.822, 33.79, 225.743, 187.315, 29.584, 45.653, 358.301, 231.174, 257.38, 146.254, 82.068, 69.208, 369.093, 245.201, 121.703, 125.837, 860.626, 203.783, 42.512, 115.779, 210.606, 321.926, 125.023, 143.462, 19.638, 43.184, 66.969, 54.852, 278.328, 115.111, 118.677, 24.842, 88.543, 29.0, 514.726, 94.274, 110.473, 185.345, 69.75, 100.387, 97.575, 18.458, 23.622, 11.04, 84.409, 97.608, 325.75, 25.877, 89.277, 160.668, 138.099, 203.579, 510.554, 102.338, 40.917, 164.256, 31.172, 133.408, 66.145, 33.562, 78.23, 9.289, 22.916, 45.386, 23.695, 21.835, 43.738, 69.091, 165.427, 860.626, 250.764, 322.664, 123.76, 454.004, 126.9, 86.944, 55.524, 288.349, 108.527, 302.21, 152.735, 121.446, 67.295, 68.474, 80.412, 97.344, 158.235, 116.296, 90.375, 103.575, 49.048, 198.611, 393.199, 118.675, 77.873, 265.574, 562.371, 677.091]


Contents of S_1.csv:
[256.435, 81.9, 74.81, 58.653, 221.698, 872.517, 144.207, 122.051, 140.213, 56.711, 596.057, 390.922, 207.

In [21]:

total_sum = 0
respective_sub_means = []
for i in range(len(total_lengths_array)):
    respective_sub_means.append(np.mean(total_lengths_array[i]))
    

# Flatten the list of arrays and calculate statistics
all_lengths = np.concatenate([np.array(arr) for arr in total_lengths_array])

# Mean, standard deviation, median, quantiles
mean_all = np.mean(all_lengths)
std_dev = np.std(all_lengths)
median_all = np.median(all_lengths)
quantiles = np.percentile(all_lengths, [25, 50, 75])
print(f"Individual sub-array means: {respective_sub_means}")
print(f"Overall mean: {mean_all}")
print(f"Standard deviation: {std_dev}")
print(f"Median: {median_all}")
print(f"Quantiles (25th, 50th, 75th): {quantiles}")
## find standard deviation, median, quantiles

Individual sub-array means: [157.96442574257426, 161.3495862068966, 101.4063969465649, 119.91225609756096, 206.39556923076924, 132.3073245614035, 73.12383189655172, 142.47756756756755]
Overall mean: 122.71612080536913
Standard deviation: 131.62299580634402
Median: 81.707
Quantiles (25th, 50th, 75th): [ 42.632    81.707   147.13075]


In [22]:
# This cell sets the view mode of matplotlib
if 1:
    %matplotlib qt
    plt.ion()
else:
    %matplotlib inline
    plt.ion()


## Pre-processing stage
 This section will do the following:
 - Convert CZI files to TIFF
 - Read the images into respective image arrays
 - Crop the image arrays to the respective region of interest


In [23]:
''' 
This cell takes in a particular input directory and output directory to convert the czi files to tiff files.
'''

main_directory = "data"
input_directory = os.path.join(main_directory, "czi_images_directory")
output_directory = os.path.join(main_directory, "tiff_images_directory")

# Check if output directory exists, create if not
if not os.path.exists(output_directory):
    print("Output Directory not found, creating one \n")
    os.makedirs(output_directory)
    print("Output Directory successfully created \n")

# Loop through files in the input directory
for filename in os.listdir(input_directory):
    full_file_path = os.path.join(input_directory, filename)
    
    # Check if it is a file and has a .czi extension
    if os.path.isfile(full_file_path) and filename.lower().endswith('.czi'):
        with czifile.CziFile(full_file_path) as czi:
            image_array = czi.asarray()
        
        # Save the image as TIFF in the output directory
        output_file_path = os.path.join(output_directory, filename + '.tiff')
        tf.imsave(output_file_path, image_array, imagej=True)
        print(f"Successfully converted: {filename} to {output_directory} \n")

Successfully converted: CTRL_INT_4.czi to data/tiff_images_directory 



/var/folders/7b/xjfqff1j40q09wj54wk3p2w80000gn/T/ipykernel_5525/1193108092.py:26: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tf.imsave(output_file_path, image_array, imagej=True)


In [24]:
def crop_image(channel_array):  
    '''
    This function takes in:
        A 2D array (channel_array which is ideally a channel of an array) 
        
    This function:
        Crops of the image to the specified x and y limits 

    This function returns:
        a cropped 2D array
    '''
    x_min = 2200
    x_max = 3200
    y_min = 1100
    y_max = 3000
    return channel_array[y_min:y_max, x_min:x_max]
    

In [25]:
#This array stores the information of the 2D arrays of the channel for statistical analysis
channel_data_array = []

In [26]:
def save_array_before_crop(image_array):
    '''
    This function takes in:
        A 2D array (image_array) 
        
    This function:
        Flattens the 2D array and sorts it numerically

    This function returns:
        The flattened and sorted 1D array (sorted_pixel_values)
    '''
    pixel_values = image_array.flatten()
    sorted_pixel_values = np.sort(pixel_values)
    return sorted_pixel_values
    

In [27]:
def split_channels(tiff_image):
    ''' 
    This function takes in:
        A image path to a tiff image (tiff_image)
        
    This function:
        Reads the image, splits the image into its distinct channels and adds those arrays to an array of arrays.

    This function returns:
        An array of arrays (image_channel_arrays)
        
    ''' 
    image_array = tf.imread(tiff_image) #Gets the image array
    image_channels_array = []
    num_channels = image_array.shape[0] # Gets the number of channels
    for channel in range(num_channels):
        channel_data = image_array[channel]  # Extract data for one channel
        channel_data_array.append(save_array_before_crop(channel_data))
        image_channels_array.append(channel_data)
    return image_channels_array
    

    

In [28]:
# This runs the command to splits the channels and saves them into the channels_directory
array = split_channels('data/tiff_images_directory/CTRL_INT_4.czi.tiff')

# Loop through the channels and save each one
for i, channel in enumerate(array):
    # Save the image using tifffile
    tf.imwrite(os.path.join("data/channels_directory", f'channel_{i+1}.tiff'), channel)

## crops and saves the images to the directory
cropped_array = [crop_image(channel) for channel in array]
for j, cropped_channel in enumerate(cropped_array):
    output_file_path = os.path.join("data/cropped_images", "cropped_image_channel_" + str(j + 1) + ".tiff")
    tf.imsave(output_file_path, cropped_channel, imagej=True)
    print(f"Successfully saved cropped channel {j + 1} to {output_file_path} \n")


Successfully saved cropped channel 1 to data/cropped_images/cropped_image_channel_1.tiff 

Successfully saved cropped channel 2 to data/cropped_images/cropped_image_channel_2.tiff 

Successfully saved cropped channel 3 to data/cropped_images/cropped_image_channel_3.tiff 

Successfully saved cropped channel 4 to data/cropped_images/cropped_image_channel_4.tiff 



/var/folders/7b/xjfqff1j40q09wj54wk3p2w80000gn/T/ipykernel_5525/3388603868.py:13: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tf.imsave(output_file_path, cropped_channel, imagej=True)


## Begin anaylsis

In this code segment we will be doing the following:
- Defining the threshhold of for the image
- Reconstructing the image (WIP)
- Removing noise and skeletizing the image

In [29]:
def reconstruction_of_image(image_array): 
    '''
    This function takes in:
        A 2D array (image_array)
    This function:
        Performs a morphological reconstruction of the 2D array
    This function returns:
        A reconstructed 2D array (enhanced_image)
    '''
    image = image_array[0]
    image = img_as_float(image)
    image_blurred = gaussian_filter(image, 1)
    seed = np.copy(image)
    seed[1:-1, 1:-1] = image.min()
    mask = image
    dilated = reconstruction(seed, mask, method='dilation')
    enhanced_image = image - dilated
    return enhanced_image
    

In [30]:
def define_threshold(channel_data_array, percentile):
    '''
    This function takes in:
        A 2D array (channel_data_array); a percentage value (percentile)
    This function:
        Calculates the threshold value.   
    This function returns:
        A threshold value (threshold_value)
    '''
    top_percentile_value = 100 - percentile
    threshold_value = np.percentile(channel_data_array[0], top_percentile_value)
    return threshold_value

In [31]:
def remove_noise(image_array):
    '''
    This function takes in:
        A 2D array (image_array); a percentage value (percentile)
    This function:
        Performs a variety of morphological operations, speciifcally
        closing, dilation, skeletonization, dilation, and skeletonization    
    This function returns:
        A 2D array that has been morphed (final_image)
    '''
    
    opened_image = morphology.closing(image_array, morphology.square(3))
    # plt.imshow(opened_image)
    dilated = morphology.dilation(opened_image, morphology.rectangle(50,1))
    # plt.imshow(~dilated)
    skel = morphology.medial_axis(opened_image, return_distance= False)
    # plt.imshow(skel)
    skeltized = morphology.skeletonize(dilated)
    # plt.imshow(skeltized)

    dilated2 = morphology.dilation(skeltized,morphology.rectangle(50,1))
    skeltized2 = morphology.skeletonize(dilated2)
    
    # cleaned = remove_small_objects(skeltized2, min_size = 2)

    final_image = opened_image
    return final_image #Returns final result of skeletized

    ##Notes: Vertically long rectangles do best, The more horizontal it is the more it will grow horizontally and connect
    ##A Square kernel initially is best to dilate and erode without losing too much. Using other shapes causises issues
    ## 
    

In [32]:
# View
plt.figure()
print(array)
removed = remove_noise(array)
original_image = tf.imread('data/channels_directory/channel_1.tiff')
overlay = (~original_image) * (~removed)
plt.imshow(overlay)

[array([[  0,   0,   0, ..., 354, 354, 354],
       [  0,   0,   0, ..., 354, 354, 354],
       [  0,   0,   0, ..., 354, 354, 354],
       ...,
       [379, 379, 379, ...,   0,   0,   0],
       [379, 379, 379, ...,   0,   0,   0],
       [379, 379, 379, ...,   0,   0,   0]], dtype=uint16), array([[  0,   0,   0, ..., 430, 430, 430],
       [  0,   0,   0, ..., 430, 430, 430],
       [  0,   0,   0, ..., 430, 430, 430],
       ...,
       [492, 492, 492, ...,   0,   0,   0],
       [492, 492, 492, ...,   0,   0,   0],
       [492, 492, 492, ...,   0,   0,   0]], dtype=uint16), array([[ 0,  0,  0, ..., 91, 91, 91],
       [ 0,  0,  0, ..., 91, 91, 91],
       [ 0,  0,  0, ..., 91, 91, 91],
       ...,
       [93, 93, 93, ...,  0,  0,  0],
       [93, 93, 93, ...,  0,  0,  0],
       [93, 93, 93, ...,  0,  0,  0]], dtype=uint16), array([[  0,   0,   0, ..., 130, 130, 130],
       [  0,   0,   0, ..., 130, 130, 130],
       [  0,   0,   0, ..., 130, 130, 130],
       ...,
       [127, 12

AttributeError: 'list' object has no attribute 'dtype'

# This section is the testing area.


In [33]:
## Local Maximums USE SKIPY Percentages

## THIS CONVERTS TO FLOAT 64

def find_local_maximums(image_array):
    '''
    This function takes in:
        A 2D array (image_array)
    This function:
        Loops through each row of the original image and uses the find_peaks() method
        in order to find the local peaks of the row.
    This function returns:
        A 2D array with 0s and the peak values at the 
        corresponding index location of the original image
    Note: the find_peaks() function will return a int64 array.
    '''
    rows = len(image_array) 
    cols = len(image_array[0])
    peaks_indices_array = []
    new_image_array = np.zeros((rows, cols), dtype=np.uint16)
    for row in range(len(image_array) - 1):
        peaks, properties = find_peaks(image_array[row], height=(1500,23000), threshold=None, distance=3, 
                                          prominence=None, width=50, wlen=None, rel_height=100, 
                                          plateau_size=None)
        prom_array = peak_prominences(image_array[row], peaks)
        peaks_indices_array.append(peaks)
    for i in range(len(image_array) - 1):
        for j in range(len(image_array[i]) - 1):
            if (j in peaks_indices_array[i]):
                new_image_array[i][j] = image_array[i][j]
    return new_image_array
    

In [34]:
## This shows the original cropped image
im = plt.imread("data/cropped_images/cropped_image_channel_1.tiff")
plt.figure()
plt.imshow(im)

In [35]:
## This section finds the local maximums of the image
plt.figure()
# print(im.dtype)
test_array = find_local_maximums(im) 
plt.imshow(test_array)



In [36]:
def percentage_change(final, original_value):
    '''
    This function calculates the percentage threshold. If the two values are within the threshold then a boolean True will
    be returned. Otherwise a boolean False will be returned.
    '''
    percentage_threshold = 50
    calculated_percent = ((final - original_value) / original_value) * 100.0
    if calculated_percent > percentage_threshold and calculated_percent >= 0:
        # print("true")
        return True
    else:
        # print("false")
        return False

In [37]:
##construct small array 
def split_array(array, pixel_location, max_distance_from_pixel_loc):
    '''
    This function takes in:
        A 2D array (array); a pixel location [pixel_location], and a maximum distance from the
        pixel location (max_distance_from_pixel_loc)
    This function:
        Splits the array at the pixel location into a left and right array.
    This function returns:
        An array of arrays, where the sub-arrays are the left and right side arrays from the pixel 
        location
    '''
    right_array = []
    left_array = []
    left_and_right = []
    # If the pixel's locations left side is negative, indicating it is on the far left
    if pixel_location - max_distance_from_pixel_loc < 0: 
        # Determines if it is on the edge of the image
        if pixel_location - max_distance_from_pixel_loc == -1 * max_distance_from_pixel_loc:
            left_array = []
        else:
            left_array = array[0 : pixel_location]
        right_array = array[pixel_location: pixel_location + max_distance_from_pixel_loc]
    # If the pixel's location right side is greater than the row length
    elif (pixel_location + max_distance_from_pixel_loc) > (len(array) - 1):
        right_array = []
        left_array = array[pixel_location - max_distance_from_pixel_loc: pixel_location-1]
    # If the pixel location is the middle of the image
    else: 
        left_array = array[pixel_location - max_distance_from_pixel_loc: pixel_location]
        right_array = array[pixel_location + 1 : pixel_location + 1 + max_distance_from_pixel_loc]
    # Appends the arrays to a final array which is returned
    left_and_right.append(left_array)
    left_and_right.append(right_array)
    return left_and_right
        
        

In [38]:
def test_algorithm(new_image_array, b, pixel_tolerance ):
    '''
    This function takes in:
        A 2D array (new_image_array); a background value (b), and a pixel tolerance (pixel_tolerance)
    This function:
        This loops through the image and determines the percentage change of the left and right side array of the corresponding
        pixel. It then determines if this pixel should be changed or set to 0. This function aims to make continous axon
        fragment lines
    This function returns:
        A 2D array which is process array.
    '''
   
    n = pixel_tolerance #pixels away no longer count maximum values
    mean_of_background = b
    non_empty_row = 0

    for i in range(len(new_image_array) - 1): ## Loops through the rows
        # print(f"row mean: {np.mean(new_image_array[i])}")
        if np.mean(new_image_array[i]) > mean_of_background: ## This is how I determine if a row is important (contains potential axons)
            # print("passed mean test")
            non_empty_row +=1 ## Increments the row count

            if non_empty_row != 1 : ## want to leave first row untouched, just want to load it                
                for j in range(len(new_image_array[i] - 1)): ## Loops through the columns
                    splitted_array = split_array(new_image_array[i-1], j, n) ## Checks the row above and splits the array into the left and right
                    combined_val = 0 # Value to calculate the combined value
                    left_side_boolean = False
                    right_side_boolean = False
                    for k in range(len(splitted_array)): # Loops through the array (size of 2)
                        if percentage_change(new_image_array[i][j], np.mean(splitted_array[k])) == True: ## If the pixel value at the specific pixel location meets the minimum threshold change 
                            combined_val += np.mean(splitted_array[k]) # Updates combined value
                            if k == 0:
                                left_side_boolean = True
                            if k == 1:
                                right_side_boolean = True
                        if k == 1:

                            if left_side_boolean == True and right_side_boolean == True :
                                new_image_array[i][j] = combined_val # Sets the original array value
                                combined_val = 0
                            elif((left_side_boolean == False and right_side_boolean == True) or
                                         (left_side_boolean == True and right_side_boolean == False)):
                                new_image_array[i][j] = 0 # Sets the original array value
                                combined_val = 0
        else:
            # print("failed mean test")
            non_empty_row = 0 ## Resets it to the 
    return new_image_array

In [39]:
plt.figure()
print(test_array.dtype)
output_array = test_algorithm(test_array,500, 10 )
print(output_array.dtype)

# final = output_array * test_array
# plt.imshow(output_array)
# print(output_array.shape)


final = test_algorithm(remove_noise(output_array * test_array),9.9**7, 10 )
plt.imshow(final)

uint16
uint16


In [64]:

def merge_lines(lines, distance_threshold):
    lines = np.squeeze(lines)
    lines = lines.tolist()

    # print (f"lines after tolist() :{lines}")
    # Helper function to check if the distance between two points is within the threshold
    
    def is_within_distance(p1, p2, threshold):
        distance = np.linalg.norm(np.array(p1) - np.array(p2))
        return distance <= threshold


    # Helper function to merge two lines into one
    def merge_two_lines(line1, line2):
        return [line1[0], line1[1], line2[2], line2[3]]
    
    def possible_merges(array,distance_threshold):
        for i in range (len(array)-1):
            x1,y1,x2,y2 = array[i] ## line one
            for j in range(i + 1, len(array)):  # Start from i + 1 to avoid redundant checks
                    x3,y3,x4,y4 =array[j]
                    if is_within_distance((x2, y2), (x3, y3), distance_threshold) or \
                    is_within_distance((x1, y1), (x4, y4), distance_threshold):
                        return True
        return False


    while possible_merges(lines, distance_threshold): ## while there are still possible merges
        print(len(lines))
        for i in range(len(lines)):
            x1, y1, x2, y2 = lines[i] ## gets line 1 
            for j in range (len(lines)):
                merged = False
                if j != i:
                    x3, y3, x4, y4 = lines[j] ## gets line 2
                    if is_within_distance((x2, y2), (x3, y3), distance_threshold) or \
                    is_within_distance((x1, y1), (x4, y4), distance_threshold) : ## end and start
                        merged_line = merge_two_lines([x1, y1, x2, y2], [x4, y4, x3, y3]) ## merge the lines
                        lines.pop(j)
                        lines.pop(i)
                        lines.append(merged_line)
                        merged = True
                        break
            if merged:
                print("merged")
                print(f"i: {i}")
                break
    return lines

   

In [65]:
## This cell below aims to detect axon fragments(continous lines) to then be able to measure them after this cell.
##WIP: Currently not working, overexposing the image.
def detect_lines (image_array):
    
    min_16bit = 0
    max_16bit = (2**16)-1

    min = 7.0 * (10**3)
    max = 8.0 * (10**3)

    normalized_min = ((min - min_16bit) / (max_16bit - min_16bit)) * 255
    normalized_max = ((max - min_16bit) / (max_16bit - min_16bit)) * 255

    normalized_image = cv.normalize(image_array, None, 0, 255, cv.NORM_MINMAX).astype(np.uint8)
    # plt.imshow(final)
    edges = cv.Canny(normalized_image, normalized_min, normalized_max)

    # merged_lines = merge_lines(edges,15)  # Merge lines

    cdst = cv.cvtColor(edges, cv.COLOR_GRAY2BGR)

    # cdst = cv.cvtColor(edges, cv.COLOR_GRAY2BGR)
    cdstP = np.copy(cdst)

    # lines = cv.HoughLines(edges,1, np.pi / 180, 200)

    linesP = cv.HoughLinesP(edges, 1, np.pi / 180, 50, None, 0, 15)

    plt.figure()
    plt.imshow(linesP)
    
    if isinstance(linesP, np.ndarray):
        print("linesP is a NumPy array.")
        print(f"before merge_lines: {linesP}")
    else:
        print("linesP is not a NumPy array.")
        
    merged_lines = merge_lines(linesP, 15)  # Merge lines

    def calculate_line_lengths(lines):
        lengths = np.sqrt((lines[:, 2] - lines[:, 0])**2 + (lines[:, 3] - lines[:, 1])**2)
        return lengths
    
    def analyze_line_lengths(lengths):
        mean_length = np.mean(lengths)
        median_length = np.median(lengths)
        std_dev = np.std(lengths)
        quantiles = np.percentile(lengths, [25, 50, 75])
    
        stats = {
            'mean': mean_length,
            'median': median_length,
            'std_dev': std_dev,
            'quantiles': quantiles
        }
        return stats
    
    line_lengths = calculate_line_lengths(merged_lines)
    print(f"Line lengths: {line_lengths}")

    # Analyze lengths
    stats = analyze_line_lengths(line_lengths)
    print(f"Mean length: {stats['mean']}")
    print(f"Median length: {stats['median']}")
    print(f"Standard deviation: {stats['std_dev']}")
    print(f"Quantiles (25th, 50th, 75th): {stats['quantiles']}")

    # Count the number of lines
    num_lines = len(merged_lines)
    print(f"Number of lines: {num_lines}")
    



    # cv.imshow("Source", image_array)
    # cv.imshow("Detected Lines (in red) - Standard Hough Line Transform", cdst)
    # cv.imshow("Detected Lines (in red) - Probabilistic Line Transform", cdstP)
    # print(f"Number of lines detected: {line_count}")
    # print(f"Lengths of detected lines: {line_lengths}")

    # plt.figure()
    # plt.imshow( image_array)
    # plt.figure()
    # plt.imshow( cdst)
    # plt.figure()
    # plt.imshow(cdstP)


In [66]:

detect_lines(final)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


linesP is a NumPy array.
before merge_lines: [[[ 617 1897  623 1826]]

 [[ 636 1701  642 1632]]

 [[ 724  628  741  432]]

 ...

 [[ 757  569  757  564]]

 [[ 390 1200  391 1197]]

 [[ 801  529  802  517]]]


KeyboardInterrupt: 